### Libraries import

In [21]:
import cv2 as cv
import numpy as np

#### Sample Video import

In [22]:
# Open the video capture
cap = cv.VideoCapture('sample_data/c&j.mp4')

In [25]:
# Read the first frame of the video
ret, frame = cap.read()

# Select the region of interest (ROI) for tracking
#r, h, c, w = 550, 20, 180, 20  # (y, height, x, width)
#roi = frame[r:r+h, c:c+w]
#
##test rectangle
#top_left = (c, r)
#bottom_right = (c+w, r+h)
#cv.rectangle(frame, top_left, bottom_right, (255,0,0), 5)

# Convert the ROI to HSV color space
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)

cv.imshow("roi", frame)
cv.waitKey(0)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [24]:
# Select the region of interest (ROI) for tracking
r, h, c, w = 550, 20, 180, 20  # (y, height, x, width)
roi = frame[r:r+h, c:c+w]

# Convert the ROI to HSV color space
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)

# Create a mask to filter out low light
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

# Compute the histogram of the ROI in the HSV color space
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])

# Normalize the histogram
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

# Define the termination criteria: either 10 iterations or moving by at least 1 pt
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

# Create the initial window for tracking
track_window = (c, r, w, h)

while True:
    # Capture a new frame
    ret, frame = cap.read()

    if ret:
        # Convert the frame to HSV color space
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

        # Back-project the histogram of the ROI onto the new frame
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        # Apply the mean shift to get the new location
        ret, track_window = cv.meanShift(dst, track_window, term_crit)

        # Draw the new tracking window on the frame
        x, y, w, h = track_window
        cv.rectangle(frame, (x, y), (x+w, y+h), 255, 2)

        # Display the result
        cv.imshow('Mean Shift Tracking', frame)

        # Exit when the user presses the Esc key
        if cv.waitKey(30) & 0xFF == 27:
            break
    else:
        break

# Release the capture and close windows
cap.release()
cv.destroyAllWindows()